In [ ]:
!pip install BeautifulSoup4
!pip install requests

In [73]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
import csv
import os

In [ ]:
# url = "https://www.jra.go.jp/datafile/seiseki/replay/2024/jyusyo.html"
url = "https://www.jra.go.jp/datafile/seiseki/replay/2024/001.html"

#変数urlのwebページからhtmlを取得
req = requests.get(url)
soup = bs(req.content, "html.parser")

elems = soup.select("div.cell.date")
race_title = elems[0].text

'2024年1月6日（土曜） 1回中山1日'

In [43]:
for num in range(1,10):
    print(f'{num:03}')

001
002
003
004
005
006
007
008
009


In [41]:
race_headers = soup.select("thead")
race_results = soup.select("tbody")
race_headers_rows = race_headers[0].find_all("tr")
rows = race_results[0].find_all("tr")
header = [cell.get_text(strip=True) for cell in race_headers[0].find_all(["th", "td"])]

data = [
    [
        cell.find('img')['src'].rsplit('/', 1)[-1].rsplit('.', 1)[0]
        if cell.has_attr('class') and 'waku' in cell['class'] and cell.find('img')
        else cell.get_text(strip=True)
        for cell in row.find_all(['th', 'td'])
    ]
    for row in rows
]


df = pd.DataFrame(data, columns=header)
csv_file = "/content/drive/MyDrive/002__プログラミング【学習】/python/raceResult.csv"
df.to_csv(csv_file, index=False, encoding="utf-8")

In [42]:
pd.read_csv(csv_file)

,着順,枠,馬番,馬名,性齢,負担重量,騎手名,タイム,着差,コーナー通過順位,推定上り,馬体重（増減）,調教師名,単勝人気
0,1,2,3,リカンカブール,牡5,56.0,津村 明秀,1:58.9,NaN,3444,34.5,494(-12),田中 克典,5
1,2,4,7,ククナ,牝6,54.0,戸崎 圭太,1:59.0,１／２,981010,34.1,468(+6),栗田 徹,8
2,3,8,15,マイネルクリソーラ,牡5,55.0,M.デムーロ,1:59.1,クビ,3464,34.6,458(+4),中野 栄治,6
3,4,4,8,ボーンディスウェイ,牡5,55.0,木幡 巧也,1:59.2,３／４,3322,34.9,490(0),牧 光二,2
4,5,6,12,ホウオウアマゾン,牡6,58.0,横山 武史,1:59.3,１／２,2222,35.1,512(-4),矢作 芳人,14
5,6,3,6,クリノプレミアム,牝7,55.5,松岡 正海,1:59.3,クビ,6777,34.7,488(+2),伊藤 伸一,10
6,7,5,10,マテンロウレオ,牡5,58.5,横山 典弘,1:59.3,ハナ,12121310,34.0,482(-2),昆 貢,3
7,8,7,13,カテドラル,牡8,58.5,荻野 極,1:59.4,３／４,14141516,33.8,488(-4),池添 学,17
8,9,6,11,ゴールデンハインド,牝4,54.0,菅原 明良,1:59.5,アタマ,1111,35.4,498(+6),武市 康男,4
9,10,3,5,マイネルファンロン,牡9,56.0,丸山 元気,1:59.5,アタマ,91077,34.8,490(+10),手塚 貴久,15


In [71]:
# base_url = "https://www.jra.go.jp/datafile/seiseki/replay/2024/jyusyo.html"
# page_data = requests.get(base_url)
# soup = bs(page_data.content, "html.parser")

# tbody = soup.select('td.result > a')
# for href in tbody:
#     text = href.get('href')
#     print(text)

AttributeError: 'str' object has no attribute 'head'

In [86]:
from google.colab import userdata


# 全体のURL
base_url = userdata.get('BASE_URL')
# レースのURL
race_base_url = userdata.get('RACE_BASE_URL')


# 全体のページから各レースページに遷移するリンクを取得
page_data = requests.get(base_url)
soup = bs(page_data.content, "html.parser")
all_links = soup.select('td.result > a')


# リンクごとに情報を取得し、CSVに書き込み
for href in all_links:
    text = href.get('href')
    result_url = race_base_url + text

    req = requests.get(result_url)
    soup = bs(req.content, "html.parser")
    # レースタイトル
    race_title = soup.select("div.cell.date")[0].text

    # レース結果
    race_headers = soup.select("thead")
    race_results = soup.select("tbody")
    race_headers_rows = race_headers[0].find_all("tr")
    rows = race_results[0].find_all("tr")
    header = ["レースタイトル"] + [cell.get_text(strip=True) for cell in race_headers[0].find_all(["th", "td"])]


    data = [
        [
            cell.find('img')['src'].rsplit('/', 1)[-1].rsplit('.', 1)[0]
            if cell.has_attr('class') and 'waku' in cell['class'] and cell.find('img')
            else cell.get_text(strip=True)
            for cell in row.find_all(['th', 'td'])
        ]
        for row in rows
    ]

    data_with_new_column = [
        [race_title] + row
        for row in data
    ]

    df = pd.DataFrame(data_with_new_column)
    csv_file = userdata.get('CSV_FILE_PATH')

    # 最初の書き込み時はヘッダーを含める
    if not os.path.exists(csv_file):
        # ファイルが存在しない場合はヘッダー付きで書き込む
        df.to_csv(csv_file, mode="w", index=False, header=True, encoding="utf-8")
    else:
        # ファイルが存在する場合はヘッダーなしで追記
        df.to_csv(csv_file, mode="a", index=False, header=False, encoding="utf-8")

In [87]:
pd.read_csv(csv_file, nrows=10)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,2024年1月6日（土曜） 1回中山1日,1,2,3,リカンカブール,牡5,56.0,津村 明秀,1:58.9,NaN,3444,34.5,494(-12),田中 克典,5
1,2024年1月6日（土曜） 1回中山1日,2,4,7,ククナ,牝6,54.0,戸崎 圭太,1:59.0,１／２,981010,34.1,468(+6),栗田 徹,8
2,2024年1月6日（土曜） 1回中山1日,3,8,15,マイネルクリソーラ,牡5,55.0,M.デムーロ,1:59.1,クビ,3464,34.6,458(+4),中野 栄治,6
3,2024年1月6日（土曜） 1回中山1日,4,4,8,ボーンディスウェイ,牡5,55.0,木幡 巧也,1:59.2,３／４,3322,34.9,490(0),牧 光二,2
4,2024年1月6日（土曜） 1回中山1日,5,6,12,ホウオウアマゾン,牡6,58.0,横山 武史,1:59.3,１／２,2222,35.1,512(-4),矢作 芳人,14
5,2024年1月6日（土曜） 1回中山1日,6,3,6,クリノプレミアム,牝7,55.5,松岡 正海,1:59.3,クビ,6777,34.7,488(+2),伊藤 伸一,10
6,2024年1月6日（土曜） 1回中山1日,7,5,10,マテンロウレオ,牡5,58.5,横山 典弘,1:59.3,ハナ,12121310,34.0,482(-2),昆 貢,3
7,2024年1月6日（土曜） 1回中山1日,8,7,13,カテドラル,牡8,58.5,荻野 極,1:59.4,３／４,14141516,33.8,488(-4),池添 学,17
8,2024年1月6日（土曜） 1回中山1日,9,6,11,ゴールデンハインド,牝4,54.0,菅原 明良,1:59.5,アタマ,1111,35.4,498(+6),武市 康男,4
9,2024年1月6日（土曜） 1回中山1日,10,3,5,マイネルファンロン,牡9,56.0,丸山 元気,1:59.5,アタマ,91077,34.8,490(+10),手塚 貴久,15
